<a href="https://colab.research.google.com/github/pallavi-allada/experiments-with-data/blob/master/Image_Classification_TransferLearning_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import keras
from keras.applications import VGG16, resnet50
from keras.preprocessing.image import load_img,ImageDataGenerator
import os
import random


Here we will not use OpenCV functions to open image or preprocess them as the images are expected to be in RGB format unlike in case of OpenCV where it is BGR. Hence lets use preprocessing functions of Keras itself.
Likewise, when we create a model object we are excluding the top which means we are excluding the fully connected layers. This helps us extract features which we will save into a CSV file for further classification.

In [0]:
TRAIN_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/classification files/train'
VALID_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/classification files/valid'
TEST_DIR = '/content/drive/My Drive/Colab Notebooks/ObjectDetection/images/classification files/test'

model = VGG16(weights='imagenet',include_top=False)

In [138]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [166]:
datagen = ImageDataGenerator(rescale=1./255)
batchsize = 4
trainSize = 240
valSize = 48
trainSize,valSize

(240, 48)

In [0]:
import numpy as np
train_features = np.zeros(shape=(trainSize,7,7,512))
train_labels = np.zeros(shape=(trainSize))
train_files = np.empty(shape=(trainSize),dtype=object)
valid_features = np.zeros(shape=(valSize,7,7,512))
valid_labels = np.zeros(shape=(valSize))
valid_files = np.empty(shape=(valSize),dtype=object)

In [168]:
train_generator = datagen.flow_from_directory(directory=TRAIN_DIR,
                                              target_size=(224,224), 
                                              batch_size=batchsize,
                                              class_mode='binary', 
                                              shuffle=True)
i=0
for images_batch, labels_batch in train_generator:
  features_batch = model.predict(images_batch)
  train_features[i*batchsize:(i+1)*batchsize]=features_batch
  train_labels[i*batchsize:(i+1)*batchsize]=labels_batch
  train_files[i*batchsize:(i+1)*batchsize]=train_generator.filenames[i*batchsize:(i+1)*batchsize]
  i+=1
  if i*batchsize >=trainSize:
    break

print(train_features.shape,type(train_features))
train_features = np.reshape(train_features,(trainSize,7*7*512))
print(train_features.shape, type(train_features))



Found 240 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


(240, 7, 7, 512) <class 'numpy.ndarray'>
(240, 25088) <class 'numpy.ndarray'>


In [169]:
valid_generator = datagen.flow_from_directory(directory=VALID_DIR,
                                              target_size=(224,224),
                                              batch_size=batchsize,
                                              class_mode='binary',
                                              shuffle=True)

i=0
for images_batch,labels_batch in valid_generator:
  print(valid_generator.filenames)
  features_batch=model.predict(images_batch)
  valid_features[i*batchsize:(i+1)*batchsize]=features_batch
  valid_labels[i*batchsize:(i+1)*batchsize]=labels_batch
  valid_files[i*batchsize:(i+1)*batchsize]=valid_generator.filenames[i*batchsize:(i+1)*batchsize]
  i+=1
  if i * batchsize >= valSize:
    break
  
valid_features = np.reshape(valid_features,(valSize,7*7*512))
print(valid_features.shape)

Found 48 images belonging to 2 classes.
['nottufted/no_tufted_headboards_130.jpg', 'nottufted/no_tufted_headboards_131.jpg', 'nottufted/no_tufted_headboards_132.jpg', 'nottufted/no_tufted_headboards_133.jpg', 'nottufted/no_tufted_headboards_134.jpg', 'nottufted/no_tufted_headboards_135.jpg', 'nottufted/no_tufted_headboards_136.jpg', 'nottufted/no_tufted_headboards_137.jpg', 'nottufted/no_tufted_headboards_138.jpg', 'nottufted/no_tufted_headboards_139.jpg', 'nottufted/no_tufted_headboards_141.jpg', 'nottufted/no_tufted_headboards_142.jpg', 'nottufted/no_tufted_headboards_144.jpg', 'nottufted/no_tufted_headboards_145.jpg', 'nottufted/no_tufted_headboards_146.jpg', 'nottufted/no_tufted_headboards_147.jpg', 'nottufted/no_tufted_headboards_148.jpg', 'nottufted/no_tufted_headboards_150.jpg', 'nottufted/no_tufted_headboards_151.jpg', 'nottufted/no_tufted_headboards_152.jpg', 'nottufted/no_tufted_headboards_153.jpg', 'nottufted/no_tufted_headboards_154.jpg', 'nottufted/no_tufted_headboards_155

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


['nottufted/no_tufted_headboards_130.jpg', 'nottufted/no_tufted_headboards_131.jpg', 'nottufted/no_tufted_headboards_132.jpg', 'nottufted/no_tufted_headboards_133.jpg', 'nottufted/no_tufted_headboards_134.jpg', 'nottufted/no_tufted_headboards_135.jpg', 'nottufted/no_tufted_headboards_136.jpg', 'nottufted/no_tufted_headboards_137.jpg', 'nottufted/no_tufted_headboards_138.jpg', 'nottufted/no_tufted_headboards_139.jpg', 'nottufted/no_tufted_headboards_141.jpg', 'nottufted/no_tufted_headboards_142.jpg', 'nottufted/no_tufted_headboards_144.jpg', 'nottufted/no_tufted_headboards_145.jpg', 'nottufted/no_tufted_headboards_146.jpg', 'nottufted/no_tufted_headboards_147.jpg', 'nottufted/no_tufted_headboards_148.jpg', 'nottufted/no_tufted_headboards_150.jpg', 'nottufted/no_tufted_headboards_151.jpg', 'nottufted/no_tufted_headboards_152.jpg', 'nottufted/no_tufted_headboards_153.jpg', 'nottufted/no_tufted_headboards_154.jpg', 'nottufted/no_tufted_headboards_155.jpg', 'nottufted/no_tufted_headboards_1

Persist the extracted train and valid features and labels to disk in a CSV file for later use in classifier


In [170]:
import pandas as pd
train_data = np.c_[train_features,train_labels,train_files]
valid_data = np.c_[valid_features,valid_labels,valid_files]

traindf = pd.DataFrame(train_data)
validdf = pd.DataFrame(valid_data)
print(traindf.shape,validdf.shape)

colnames=[]
for i in range(0,train_data.shape[1]):
  colnames.append("col"+str(i))

traindf.to_csv('train_data.csv',header=colnames,index=False)
validdf.to_csv('valid_data.csv',header=colnames,index=False)


(240, 25090) (48, 25090)


In [172]:
validdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,25050,25051,25052,25053,25054,25055,25056,25057,25058,25059,25060,25061,25062,25063,25064,25065,25066,25067,25068,25069,25070,25071,25072,25073,25074,25075,25076,25077,25078,25079,25080,25081,25082,25083,25084,25085,25086,25087,25088,25089
0,0,0,0,0,0.214273,0,0,0,0,0,0,0,0,0,0,0.698737,0,0.0891224,0,0,0.445141,0,0,0,0,0,1.47093,0,0,0,0.336809,0,0,0,0.471808,0,0,0,0,0,...,0.172736,0,0,0,0,0,1.2257,0,0.130577,0.157227,0,0,0.0244498,0,0,0,0,0.516854,0,0,1.01911,0.209477,0,0,0,0.286374,0.0861735,0,0.870428,0.437696,0,0,0,0.177616,0.0465111,0,0.932993,0,0,nottufted/no_tufted_headboards_130.jpg
1,0.487019,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.15877,0,0,0,0,0,0,0,0,0,0,0.778481,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.688911,0,0,0.0490257,0.108414,0,0.318886,0,0.415205,0,0,0,0,0,0.652403,0,0.197331,0.38368,0,0,0.667737,0.812483,0,0.519777,0.486104,0.187144,0.537642,0,0.82271,0.198006,0.260735,0.941922,0.0788686,0.138461,0,0.945491,1.16626,0,0,nottufted/no_tufted_headboards_131.jpg
2,0.0893167,0,0,0,1.63863,0,0,0,0,0,0,0,0,0,0,0,0,0.319061,0,0,0,0.5604,0,0,0,0,1.15747,0.0328545,0,0,0,0,0,0,0,0,0,0,0,0,...,0.748606,0,0,0,0,0,0.0974676,0,0,0,0,0,0.186948,0,0,0,0,0.183685,0,0,0.195178,0.00174294,0,0,0,0,0.7319,0,1.42064,0,0,0,0,0,0.100712,0,0.77016,0,1,nottufted/no_tufted_headboards_132.jpg
3,0.131593,0,0,0,0.837714,0,0,0,0,0,0,0,0,0,0,0.332513,0,0.64801,0,0,0,0,0,0,0,0,0.350117,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.769669,0,0,0,0,0,0.516228,0,0.364248,0,0,0,0,0,0,0,0,1.76237,0,0,0,0,0.26407,0,0,0,0,0,0.760217,0.233183,0.527352,0.365341,0.834831,0.297977,0,0,0.382748,0,1,nottufted/no_tufted_headboards_133.jpg
4,0.0448531,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.501706,0,0,0,0,0,0,0,0,0.224301,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.660825,0,0,0,0,0.050256,1.14065,0,0,0,0,0,0.239693,0,0,0,0,0.614743,0,0,0.441456,0.445654,0,0,0,0,0,0,1.66761,0.117692,0.389394,0,0,0.648803,0.293824,0,0.687846,0,0,nottufted/no_tufted_headboards_134.jpg
5,0.00378838,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0721922,0,0.832496,0,0,0,0,0,0,0,0,1.67747,0,0.280124,0,0,0,0,0,0.279499,0,0,0,0,0,...,0.36446,0,0.206794,0,0,0,0.146785,0,0,0.427725,0,0,0,0,0,0,0,0,0.419708,0,1.76414,0,0,0,0,0,0,0,1.34684,0.0169922,0.0514224,0,0,0.743036,0,0,1.26859,0,1,nottufted/no_tufted_headboards_135.jpg
6,0.0793164,0,0.498505,0,0,0,0,0,0,0,0,0,0.599454,0,0,0.12809,0,1.00706,0,0,0,0,0,0,0,0,0.218016,0,1.11,0,0.390491,0,0.0194243,0,0.347863,0,0,0,0,0,...,1.08386,0,0,0,0,0,0.714845,0,0.244749,0,0,0,0.577929,0,0,0,0,0.345363,0,0,2.57326,0.288763,0,0,0,0,0.193712,0,1.78922,0.164751,0.163875,0,0,1.10865,0.174959,0,1.10613,0,1,nottufted/no_tufted_headboards_136.jpg
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.636926,0,0.350291,0,0,0,0,0,0,0,0,0.336315,0,0,0,0,0,0.0731813,0,0,0,0,0,0,0,...,0.610076,0,0,0,0,0,0.736263,0.150653,0.771645,0,0,0,0.0941401,0,0,0,0,0.748679,0.811298,0,1.61037,0,0,0,0.0165622,0.145092,0.556929,0,2.029,0,0.403171,0,0,0.717837,0,0,0.450972,0,1,nottufted/no_tufted_headboards_137.jpg
8,0.0993626,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.376911,0,0.736284,0,0,0,0,0,0,0,0,1.01021,0,0,0,0.0113643,0,0,0,0.279432,0,0,0.176259,0,0,...,0.356766,0,0,0,0,0,0.274446,0,0,0,0,0,0.311713,0,0,0,0,0.485569,0,0,0.186255,0.0266291,0,0,0,0,0,0,1.52959,0,0,0,0,0.434573,0,0,1.18162,0,0,nottufted/no_tufted_headboards_138.jpg
9,0,0,0,0,0.405805,0,0,0,0,0,0,0,0.734808,0,0,0,0,0.723786,0,0,0,0,0,0,0,0,0.937991,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.566541,0,0,0,0,0,0.803082,0,0,0,0,0,0,0,0,0,0,0.111637,0,0,0.212073,0,0,0,0,0.131954,0.297692,0,1.53364,0,0,0,0.0733133,0.410301,0,0,1.00023,0,0,nottufted/no_tufted_headboards_139.jpg


Read data from train and valid CSV files and extract the features and labels.

In [0]:
#Read train and valid csv data anf create a classifier
import pandas as pd
train_data = pd.read_csv('train_data.csv')
valid_data = pd.read_csv('valid_data.csv')
train_features = train_data[train_data.columns.difference(['col25088','col25089'])]
train_labels = train_data['col25088']
valid_features = valid_data[valid_data.columns.difference(['col25088','col25089'])]
valid_labels = valid_data['col25088']

(trainX, trainY) = (train_features,train_labels)
(validX,validY) = (valid_features,valid_labels)

Using sklearn Logistic Regression to fit the model

In [174]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
model = LogisticRegression(solver='lbfgs')
model.fit(trainX,trainY)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [175]:
print(validY.values)
preds = model.predict(validX)
print(preds)
print(classification_report(validY, preds))
confusion_matrix(validY,preds)

[0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0.]
[1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0.]
              precision    recall  f1-score   support

         0.0       0.95      0.83      0.89        24
         1.0       0.85      0.96      0.90        24

    accuracy                           0.90        48
   macro avg       0.90      0.90      0.90        48
weighted avg       0.90      0.90      0.90        48



array([[20,  4],
       [ 1, 23]])

Approach 2 - 
Creating a sequential network to train the extracted features

In [0]:
from keras import models
from keras import layers
from keras import optimizers

In [80]:
class_model = models.Sequential()
class_model.add(layers.Dense(256,activation='relu',input_dim=(7*7*512)))
class_model.add(layers.Dropout(rate=0.5))
class_model.add(layers.Dense(128,activation='relu'))
class_model.add(layers.Dropout(0.5))
# class_model.add(layers.Dense(64,activation='relu'))
# class_model.add(layers.Dropout(0.5))
class_model.add(layers.Dense(1,activation='sigmoid'))

W0826 06:52:23.729173 140703808083840 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [81]:
class_model.compile(optimizer=optimizers.RMSprop(lr=2e-04),
             loss='binary_crossentropy',
             metrics=['accuracy'])

class_model.summary()


W0826 06:52:24.765819 140703808083840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 06:52:24.777408 140703808083840 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 6,455,809
Trainable params: 6,455,809
Non-trainable params: 0
_________________________________________________________________


In [82]:
history = class_model.fit(train_features,
                    train_labels,
                    epochs=30,
                    batch_size=4,
                    validation_data=(valid_features,valid_labels))

Train on 240 samples, validate on 48 samples
Epoch 1/30
240/240 [==============================] - 2s 8ms/step - loss: 1.0165 - acc: 0.5583 - val_loss: 0.6274 - val_acc: 0.5417
Epoch 2/30
240/240 [==============================] - 1s 3ms/step - loss: 0.6868 - acc: 0.7042 - val_loss: 0.3806 - val_acc: 0.8542
Epoch 3/30
240/240 [==============================] - 1s 3ms/step - loss: 0.5774 - acc: 0.7667 - val_loss: 0.3399 - val_acc: 0.8958
Epoch 4/30
240/240 [==============================] - 1s 3ms/step - loss: 0.3773 - acc: 0.8458 - val_loss: 0.3786 - val_acc: 0.8125
Epoch 5/30
240/240 [==============================] - 1s 3ms/step - loss: 0.3852 - acc: 0.8417 - val_loss: 0.2581 - val_acc: 0.9167
Epoch 6/30
240/240 [==============================] - 1s 3ms/step - loss: 0.3160 - acc: 0.8750 - val_loss: 0.2694 - val_acc: 0.8958
Epoch 7/30
240/240 [==============================] - 1s 3ms/step - loss: 0.2498 - acc: 0.9042 - val_loss: 0.3553 - val_acc: 0.8958
Epoch 8/30
240/240 [===========

In [83]:
fnames = valid_generator.filenames
ground_truth = valid_generator.classes
 
label2index = valid_generator.class_indices
print(label2index)
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
predictions = class_model.predict_classes(valid_features)
prob = class_model.predict(valid_features)
correct = 0

i=0
for itm in ground_truth:
  if(itm == predictions[i][0]):
    correct+=1
  i+=1
print(correct)


{'nottufted': 0, 'tufted': 1}
27
